In [3]:
import os
import pandas as pd
import numpy as np
import re
import sys
from collections import Counter
import itertools
scripts_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))
sys.path.insert(0, scripts_dir)
import data_processing as pr



In [4]:
#define paths and names
data_paths = pr.get_datapaths('HousePrices')
data_in, data_info = pr.get_data(data_paths, print_output=True, split='train')

loading data and info...
data info: 
MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	       Agriculture
       C (all)	       Commercial
       FV	       Floating Village Residential
       I	       Industrial
       RH	       Residential High Density
     

In [5]:
options_dict = pr.parse_info(data_info, print_output=True)

parsing data info file for data options...
number of descriptions:  79
numbers of categories:  79
options for each category
MSSubClass: ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
MSZoning: ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
LotFrontage: []
LotArea: []
Street: ['Grvl', 'Pave', ' ']
Alley: ['Grvl', 'Pave', 'NA']
LotShape: ['Reg', 'IR1', 'IR2', 'IR3']
LandContour: ['Lvl', 'Bnk', 'HLS', 'Low']
Utilities: ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']
LotConfig: ['Inside', 'Corner', 'CulDSac', 'FR2', 'FR3']
LandSlope: ['Gtl', 'Mod', 'Sev']
Neighborhood: ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker']
Condition1: ['Artery', 'Feedr', 'Norm', 'RRNn', 'RRAn', 'PosN', 'PosA', 'RRNe', 'RRAe']
Condition2: ['Artery', 

In [6]:
#different methods of transforming data
scalar_inputs = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',\
                 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'GarageYrBlt', 'GarageCars', 'GarageArea',\
                 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'YrSold', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',\
                    'KitchenAbvGr', 'Fireplaces']
to_scalar = ['LotShape', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'CentralAir', 'Functional',\
             'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'KitchenQual', 'FireplaceQu', 'HeatingQC']
one_hot = ['MSSubClass', 'MSZoning', 'Street', 'LotConfig', 'Neighborhood', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'Foundation',\
            'SaleType', 'SaleCondition']
one_hot_plus = {'Alley': {'NA': 0}, 'LandContour': {'Lvl': 0}, 'MasVnrType': {'None': 0, 'NA': 'dist', 'weight': 'MasVnrArea'}, 'GarageType':{'NA': 0},\
                 'MiscFeature': {'NA': 0, 'weight': 'MiscVal'}, 'Heating': {'weight': 'HeatingQC'}}
case_by_case =  {'Utilities': [{'AllPub':[1, 1, 1], 'NoSewr':[0, 1, 1], 'NoSeWa':[0, 0, 1], 'ELO':[0, 0, 0]}, ['Sew', 'Wat', 'Gas']],
           'Condition1': [{'Artery':[1, 0, 0, 0], 'Feedr':[0.5, 0, 0, 0], 'Norm': [0, 0, 0, 0], 'RRNn': [0, 0.5, 0, 0],
                           'RRAn':[0, 1, 0, 0], 'PosN':[0, 0, 0, .5], 'PosA': [0, 0, 0, 1], 'RRNe':[0, 0, 0.5, 0], 'RRAe': [0, 0, 1, 0]},
                           ['Road', 'NS_rail', 'EW_rail', 'Positive']],#add to Condition2
            'Condition2': 'Condition1',
            'BldgType': [{'1Fam':[0, 0], '2fmCon':[1, 0], 'Duplex': [0.5, 0], 'TwnhsE':[0, 0.5], 'TwnhsI': [0, 1], 'Twnhs':[0, 1]},
                           ['2fam', 'Twnhs']],
            'HouseStyle': [{'1Story': [0, 0, 0], '1.5Fin': [0.4, 0, 0], '1.5Unf': [0.2, 0, 0], '2Story': [0.6, 0, 0], '2.5Fin':[1, 0, 0],
                            '2.5Unf':[0.8, 0, 0], 'SFoyer': [0, 1, 0], 'SLvl': [0, 0, 1]}, ['Stories', 'SFoyer', 'SLvl']],
            'BsmtFinType1': [{'NA': [0], 'Unf': [0], 'LwQ':[0.2], 'Rec':[0.4], 'BLQ':[0.6], 'ALQ':[0.8], 'GLQ':[1]},
                             ['BsmtFiQual1']],
       	    'BsmtFinType2': [{'NA': [0], 'Unf': [0], 'LwQ':[0.2], 'Rec':[0.4], 'BLQ':[0.6], 'ALQ':[0.8], 'GLQ':[1]},
                             ['BsmtFiQual2']],
            'Electrical': [{'SBrkr':[1, 0, 0], 'FuseA':[2/3, 0, 0], 'FuseF':[1/3, 0, 0], 'FuseP':[0, 0, 0], 'Mix':[0, 1, 0], 'NA':[0, 0, 1]},
                           ['BreakWire_rate', 'Mixed', 'NA']]
                           }
transformations = {'scalar_inputs': scalar_inputs, 'to_scalar': to_scalar, 'one_hot': one_hot, 'one_hot_plus': one_hot_plus, 'case_by_case': case_by_case}
#post processing
#delete TotalBsmtSF
#do to_scalar transformation and replace cols in data_in
#transform MoSold and add to year
#transform one hot, one hot plus and case to case entries. Store in a dictionary.
#Change NA in Electrical to dist of the other entries. Remove NA
#combine the transformed entries in the dict back into the original dataframe. Check for repeats
#take mean and var of each feature and scale and renormalize

In [7]:
data_in, options_dict = pr.change_types(data_in, options_dict, transformations, print_output=True)

resolving types in options vs. data...
Intermediate result for options_dict:
MSSubClass: ['20', '30', '40', '45', '50', '60', '70', '75', '80', '85', '90', '120', '150', '160', '180', '190']
type of options:  <class 'str'>
type of actual:  <class 'str'>
MSZoning: ['A', 'C (all)', 'FV', 'I', 'RH', 'RL', 'RP', 'RM']
type of options:  <class 'str'>
type of actual:  <class 'str'>
LotFrontage: {'variable_type': <class 'str'>}
type of actual:  <class 'str'>
LotArea: {'variable_type': <class 'int'>}
type of actual:  <class 'numpy.int64'>
Street: ['Grvl', 'Pave', ' ']
type of options:  <class 'str'>
type of actual:  <class 'str'>
Alley: ['Grvl', 'Pave', 'NA']
type of options:  <class 'str'>
type of actual:  <class 'str'>
LotShape: ['Reg', 'IR1', 'IR2', 'IR3']
type of options:  <class 'str'>
type of actual:  <class 'str'>
LandContour: ['Lvl', 'Bnk', 'HLS', 'Low']
type of options:  <class 'str'>
type of actual:  <class 'str'>
Utilities: ['AllPub', 'NoSewr', 'NoSeWa', 'ELO']
type of options:  <cl

In [8]:
#count keys to make sure they add up to the right number
full_list = one_hot + scalar_inputs +  to_scalar + list(case_by_case.keys()) + list(one_hot_plus.keys()) + ['MasVnrArea', 'MiscVal', 'MoSold']
for item in full_list:
    if item not in options_dict.keys():
        print(item)
his = Counter(full_list)
for k,v in his.items():
    if v>1:
        print(f'key {k} has more than one occurance')


In [9]:
def add_to_series(old_series, pos, new_label, val):
    new_index  = old_series.index.insert(pos, new_label)
    # insert into the underlying array of values
    new_values = np.insert(old_series.values, pos, val)
    new_series = pd.Series(new_values, index=new_index)
    return new_series

def input_dist(df_new, key, dist=None):
    if dist is None:
        dist = df_new[df_new[key]!=1.].mean() #find distribution of different values excluding unknowns
    else:
        dist = add_to_series(dist, df_new.columns.get_loc(key), key, 0) #add an entry to dist for key
    df_new[df_new[key]==1] = dist #Give the unknowns the distribution of inputs
    #drop unknown column from df_new and dist
    df_new = df_new.drop(columns=[key])
    dist = dist.drop(index=key)
    return df_new, dist

def to_one_hot(data_in, categ, options, dtype=float):
    num_data = len(data_in)
    one_col = data_in[categ] #extract the column we want to transform
    actual_opts = list(set(one_col.tolist())) #extract possible values entries in that column have
    #print(actual_opts)
    to_add = []
    for item in options:
        #add missing entries
        if item not in actual_opts:
            one_col.loc[len(one_col)] = item  # adding a row
    df1 = pd.get_dummies(one_col, dtype=dtype) #transform to one hot encoding
    df1 = df1.loc[:num_data-1] #remove extra rows
    return df1

def to_one_hot_plus(data_in, categ, options, conv_dict, dist=None):
    df_new = to_one_hot(data_in, categ, options) #transform to one hot encoding
    #consider special cases
    for k2, val in conv_dict.items():
        if val==0:
            df_new = df_new.drop(columns=[k2])
        elif val=='dist':
            df_new, dist = input_dist(df_new, k2, dist=dist)
        elif k2=='weight':
            df_new = df_new.mul(data_in[val], axis=0)
    if dist is not None:
        dist = dist[dist.index.isin(df_new.columns)]

    return df_new, dist

def create_mappings(to_scalar, options_dict):
    mappings = {}
    for k in to_scalar:
        num_opts = len(options_dict[k])
        
        # define mapping in descending order from 1 to 0.
        mappings[k] = {opt: 1-i/(num_opts-1) for i, opt in enumerate(options_dict[k])}
    return mappings

def cat_to_vec(init_df, mapping, new_cols=None, cat=None):
    if cat is None:
        cat = init_df.columns[0]
    vec = init_df[cat].map(mapping)

    # turn list‐values into real columns
    vec_df = pd.DataFrame(vec.tolist(),
                    index=init_df.index,
                    columns=new_cols)
    return vec_df

In [10]:
if 'MSSubClass' in options_dict.keys():
    print('yes')

yes


In [11]:
data_in = pr.level_transform(data_in, options_dict, transformations['to_scalar'], print_output=True)

performing level transformations...
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond

In [12]:
data_in = pr.convert_to_str(data_in, transformations['one_hot'] + list(transformations['one_hot_plus'].keys()))

In [13]:
#convert one hot and one hot plus
dfs, dists = pr.create_new_cols(data_in, options_dict, transformations)


performing one_hot_plus transformations...
performing one_hot transformations...
performing case by case transformations...


/Users/mitchellweikert/Documents/random_projects/HousePrices/scripts/data_processing.py:149: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0009149130832570906' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_new[df_new[key]==1] = dist #Give the unknowns the distribution of inputs


In [14]:
data_in = pr.update_data(data_in, dfs, print_output=True)

data columns before removal: 
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
   

In [15]:
print(data_in)

      LotFrontage  LotArea  LotShape  LandSlope  OverallQual  OverallCond  \
0            64.0     6979  1.000000        1.0            6            5   
1            72.0    10791  1.000000        1.0            4            5   
2            84.0    11340  1.000000        1.0            6            5   
3            60.0    10800  1.000000        1.0            4            4   
4            64.0    13053  1.000000        1.0            6            7   
...           ...      ...       ...        ...          ...          ...   
1089         71.0     9520  0.666667        1.0            8            5   
1090         71.0     9204  1.000000        1.0            5            5   
1091         90.0     8100  1.000000        1.0            5            5   
1092         70.0     9135  1.000000        1.0            6            5   
1093         63.0     7875  1.000000        1.0            7            5   

      YearBuilt  YearRemodAdd  MasVnrArea  ExterQual  ...  \
0          198

In [16]:
data_in, stats = pr.regularize(data_in)

In [17]:
print(data_in)

      LotFrontage   LotArea  LotShape  LandSlope  OverallQual  OverallCond  \
0        0.204984 -0.334616  0.716459   0.230947    -0.082722    -0.525484   
1        0.434165  0.019272  0.716459   0.230947    -1.542356    -0.525484   
2        0.777935  0.070238  0.716459   0.230947    -0.082722    -0.525484   
3        0.090394  0.020107  0.716459   0.230947    -1.542356    -1.432235   
4        0.204984  0.229264  0.716459   0.230947    -0.082722     1.288017   
...           ...       ...       ...        ...          ...          ...   
1089     0.405517 -0.098722 -1.013798   0.230947     1.376913    -0.525484   
1090     0.405517 -0.128058  0.716459   0.230947    -0.812539    -0.525484   
1091     0.949821 -0.230548  0.716459   0.230947    -0.812539    -0.525484   
1092     0.376870 -0.134463  0.716459   0.230947    -0.082722    -0.525484   
1093     0.176337 -0.251435  0.716459   0.230947     0.647096    -0.525484   

      YearBuilt  YearRemodAdd  MasVnrArea  ExterQual  ...  \
0 

: 